In [242]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

import rdkit
from rdkit import Chem, DataStructs
from rdkit.Chem import Draw, rdmolops, AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors


from scipy.stats import pearsonr


# sklearn ML models
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn import svm

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import  LGBMRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.model_selection import KFold
import joblib
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity='all'


In [243]:
df = pd.read_csv('../01-database-preprocessing-1203dp-to-1115dp/raw/atom_number_wH_sort_1115-backbone-correction-newSMILES.csv')
df.head()

,Nickname,bandgap(eV),c_smiles,newSMILES,Ref.No
0,P3HT,1.93,CCCCCCc1cc(C)sc1C,Cc1cc(C)c(s1)-c1cc(C)c(s1)-c1cc(C)c(s1)-c1cc(C...,S10
1,P3HST,1.82,CCCCCCSc1cc(C)sc1C,CSc1cc(sc1C)-c1sc(cc1SC)-c1sc(cc1SC)-c1sc(cc1S...,S123
2,POPT,1.76,CCCCCCCCc1ccc(-c2cc(C)sc2C)cc1,Cc1cc(c(s1)-c1cc(c(s1)-c1cc(c(s1)-c1cc(c(s1)-c...,S126
3,PT-C1,1.92,CCCCC(CC)COC(=O)c1cc(C)sc1-c1ccc(C)s1,COC(=O)c1cc(C)sc1-c1ccc(s1)-c1cc(C(=O)OC)c(s1)...,S122
4,PT-C2,1.89,CCCCC(CC)COC(=O)c1cc(C)sc1-c1ccc(-c2ccc(C)s2)s1,COC(=O)c1cc(C)sc1-c1ccc(s1)-c1ccc(s1)-c1cc(C(=...,S122


# exp HOMO-LUMO gap

In [244]:
df_exp = df['bandgap(eV)']
df_exp

0       1.93
1       1.82
2       1.76
3       1.92
4       1.89
        ... 
1110    1.68
1111    1.65
1112    1.66
1113    1.52
1114    1.73
Name: bandgap(eV), Length: 1115, dtype: float64

# RDKit-209 features

In [245]:
rdkit_feature = pd.read_csv('monomer-1115dp-RDKit209.csv')
rdkit_feature

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,2.353121,2.353121,1.287870,1.287870,0.608740,196.359,176.199,196.128572,74,0,...,0,0,0,0,0,0,0,1,2,0
1,2.323896,2.323896,1.288704,1.288704,0.485488,228.426,208.266,228.100643,80,0,...,1,0,0,0,0,0,0,1,3,0
2,2.313310,2.313310,1.230210,1.230210,0.463323,300.511,272.287,300.191172,114,0,...,0,0,0,0,0,0,0,1,4,0
3,12.542211,12.542211,0.175600,-0.175600,0.510163,350.549,324.341,350.137422,126,0,...,0,0,0,0,0,0,0,2,0,0
4,12.817589,12.817589,0.184059,-0.184059,0.318494,432.676,404.452,432.125143,150,0,...,0,0,0,0,0,0,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,5.448759,5.448759,0.613257,0.613257,0.035659,2093.634,1894.050,2091.310291,788,0,...,0,0,0,0,0,0,0,7,52,0
1111,6.964899,6.964899,0.506884,-0.861916,0.035381,2097.349,1897.765,2095.457524,820,0,...,0,0,0,0,0,0,0,2,48,0
1112,7.004105,7.004105,0.497358,-0.874785,0.035381,2261.603,2057.987,2259.432966,868,0,...,0,0,0,0,0,0,0,4,48,0
1113,17.056452,17.056452,0.015571,0.015571,0.035580,2289.808,2066.032,2287.561538,886,0,...,0,0,0,0,0,0,0,6,56,0


# drop sp3-N polymers + donor-692

In [246]:
# non_alkyl_idx = [46,49,68,79,202,210,217,252,255,262,273,274,318,355,358,375,441,
#                 455,810,812,914,932,934,937,947,1007]

sp3_N_list = [  24,   44,  191,  201,  206,  209,  251,  317,  318,  332,  374,
             381,  388,  454,  913,  931,  936, 1006]
drop_list = sp3_N_list+[691]
print('Total data points: ', 1115-len(drop_list))

Total data points:  1096


In [247]:
df_exp = df_exp[~df_exp.index.isin(drop_list)].reset_index(drop=True)
rdkit_feature = rdkit_feature[~rdkit_feature.index.isin(drop_list)].reset_index(drop=True)


In [248]:
rdkit_feature

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,2.353121,2.353121,1.287870,1.287870,0.608740,196.359,176.199,196.128572,74,0,...,0,0,0,0,0,0,0,1,2,0
1,2.323896,2.323896,1.288704,1.288704,0.485488,228.426,208.266,228.100643,80,0,...,1,0,0,0,0,0,0,1,3,0
2,2.313310,2.313310,1.230210,1.230210,0.463323,300.511,272.287,300.191172,114,0,...,0,0,0,0,0,0,0,1,4,0
3,12.542211,12.542211,0.175600,-0.175600,0.510163,350.549,324.341,350.137422,126,0,...,0,0,0,0,0,0,0,2,0,0
4,12.817589,12.817589,0.184059,-0.184059,0.318494,432.676,404.452,432.125143,150,0,...,0,0,0,0,0,0,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,5.448759,5.448759,0.613257,0.613257,0.035659,2093.634,1894.050,2091.310291,788,0,...,0,0,0,0,0,0,0,7,52,0
1092,6.964899,6.964899,0.506884,-0.861916,0.035381,2097.349,1897.765,2095.457524,820,0,...,0,0,0,0,0,0,0,2,48,0
1093,7.004105,7.004105,0.497358,-0.874785,0.035381,2261.603,2057.987,2259.432966,868,0,...,0,0,0,0,0,0,0,4,48,0
1094,17.056452,17.056452,0.015571,0.015571,0.035580,2289.808,2066.032,2287.561538,886,0,...,0,0,0,0,0,0,0,6,56,0


# Feature engineering

## 1. remove missing value

In [249]:
nan_counts = rdkit_feature.isna().sum()
nan_counts[nan_counts>0]

MaxPartialCharge       47
MinPartialCharge       47
MaxAbsPartialCharge    47
MinAbsPartialCharge    47
BCUT2D_MWHI            60
BCUT2D_MWLOW           60
BCUT2D_CHGHI           60
BCUT2D_CHGLO           60
BCUT2D_LOGPHI          60
BCUT2D_LOGPLOW         60
BCUT2D_MRHI            60
BCUT2D_MRLOW           60
dtype: int64

In [250]:
nan_columns = rdkit_feature.columns[rdkit_feature.isna().any()].tolist()
nan_columns

['MaxPartialCharge',
 'MinPartialCharge',
 'MaxAbsPartialCharge',
 'MinAbsPartialCharge',
 'BCUT2D_MWHI',
 'BCUT2D_MWLOW',
 'BCUT2D_CHGHI',
 'BCUT2D_CHGLO',
 'BCUT2D_LOGPHI',
 'BCUT2D_LOGPLOW',
 'BCUT2D_MRHI',
 'BCUT2D_MRLOW']

In [251]:
rdkit_feature[nan_columns].loc[246]

MaxPartialCharge            inf
MinPartialCharge      -0.139029
MaxAbsPartialCharge         inf
MinAbsPartialCharge    0.139029
BCUT2D_MWHI                 NaN
BCUT2D_MWLOW                NaN
BCUT2D_CHGHI                NaN
BCUT2D_CHGLO                NaN
BCUT2D_LOGPHI               NaN
BCUT2D_LOGPLOW              NaN
BCUT2D_MRHI                 NaN
BCUT2D_MRLOW                NaN
Name: 246, dtype: float64

In [252]:
# df_drop_nan = rdkit_feature.dropna(axis=1)
# df_drop_nan

## 2. remove infinite values

In [253]:
inf_mask = np.isinf(rdkit_feature)
inf_columns = rdkit_feature.columns[inf_mask.any()].tolist()
inf_columns

['MaxPartialCharge',
 'MinPartialCharge',
 'MaxAbsPartialCharge',
 'MinAbsPartialCharge']

In [254]:
rdkit_feature[inf_columns].loc[246]

MaxPartialCharge            inf
MinPartialCharge      -0.139029
MaxAbsPartialCharge         inf
MinAbsPartialCharge    0.139029
Name: 246, dtype: float64

In [255]:
drop_list = list(set(nan_columns) | set(inf_columns))
drop_list

['BCUT2D_MWHI',
 'BCUT2D_CHGHI',
 'BCUT2D_MRLOW',
 'MaxPartialCharge',
 'BCUT2D_MRHI',
 'BCUT2D_CHGLO',
 'BCUT2D_MWLOW',
 'MaxAbsPartialCharge',
 'MinAbsPartialCharge',
 'BCUT2D_LOGPHI',
 'BCUT2D_LOGPLOW',
 'MinPartialCharge']

In [256]:
df_rdkit_screen = rdkit_feature.drop(drop_list, axis = 1)
df_rdkit_screen

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,2.353121,2.353121,1.287870,1.287870,0.608740,196.359,176.199,196.128572,74,0,...,0,0,0,0,0,0,0,1,2,0
1,2.323896,2.323896,1.288704,1.288704,0.485488,228.426,208.266,228.100643,80,0,...,1,0,0,0,0,0,0,1,3,0
2,2.313310,2.313310,1.230210,1.230210,0.463323,300.511,272.287,300.191172,114,0,...,0,0,0,0,0,0,0,1,4,0
3,12.542211,12.542211,0.175600,-0.175600,0.510163,350.549,324.341,350.137422,126,0,...,0,0,0,0,0,0,0,2,0,0
4,12.817589,12.817589,0.184059,-0.184059,0.318494,432.676,404.452,432.125143,150,0,...,0,0,0,0,0,0,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,5.448759,5.448759,0.613257,0.613257,0.035659,2093.634,1894.050,2091.310291,788,0,...,0,0,0,0,0,0,0,7,52,0
1092,6.964899,6.964899,0.506884,-0.861916,0.035381,2097.349,1897.765,2095.457524,820,0,...,0,0,0,0,0,0,0,2,48,0
1093,7.004105,7.004105,0.497358,-0.874785,0.035381,2261.603,2057.987,2259.432966,868,0,...,0,0,0,0,0,0,0,4,48,0
1094,17.056452,17.056452,0.015571,0.015571,0.035580,2289.808,2066.032,2287.561538,886,0,...,0,0,0,0,0,0,0,6,56,0


## 3. delete columns with constant values

In [257]:
constant_columns = rdkit_feature.columns[rdkit_feature.nunique() <= 1]
constant_columns

Index(['NumRadicalElectrons', 'SMR_VSA8', 'SlogP_VSA9',
       'NumSaturatedCarbocycles', 'fr_Al_COO', 'fr_Al_OH', 'fr_Al_OH_noTert',
       'fr_ArN', 'fr_Ar_COO', 'fr_Ar_OH', 'fr_COO', 'fr_COO2', 'fr_HOCCN',
       'fr_NH2', 'fr_N_O', 'fr_Ndealkylation2', 'fr_SH', 'fr_aldehyde',
       'fr_alkyl_carbamate', 'fr_amidine', 'fr_azide', 'fr_azo', 'fr_barbitur',
       'fr_benzodiazepine', 'fr_diazo', 'fr_dihydropyridine', 'fr_epoxide',
       'fr_guanido', 'fr_hdrzine', 'fr_hdrzone', 'fr_imidazole', 'fr_isocyan',
       'fr_isothiocyan', 'fr_lactam', 'fr_lactone', 'fr_morpholine',
       'fr_nitro', 'fr_nitro_arom', 'fr_nitro_arom_nonortho', 'fr_nitroso',
       'fr_oxime', 'fr_phenol', 'fr_phenol_noOrthoHbond', 'fr_phos_acid',
       'fr_phos_ester', 'fr_piperdine', 'fr_piperzine', 'fr_priamide',
       'fr_prisulfonamd', 'fr_quatN', 'fr_term_acetylene', 'fr_tetrazole',
       'fr_thiocyan', 'fr_urea'],
      dtype='object')

In [258]:
df_rdkit_screen = df_rdkit_screen.drop(constant_columns, axis = 1)
df_rdkit_screen

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,FpDensityMorgan1,...,fr_nitrile,fr_oxazole,fr_para_hydroxylation,fr_pyridine,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_thiazole,fr_thiophene,fr_unbrch_alkane
0,2.353121,2.353121,1.287870,1.287870,0.608740,196.359,176.199,196.128572,74,1.153846,...,0,0,0,0,0,0,0,0,1,2
1,2.323896,2.323896,1.288704,1.288704,0.485488,228.426,208.266,228.100643,80,1.214286,...,0,0,0,0,1,0,0,0,1,3
2,2.313310,2.313310,1.230210,1.230210,0.463323,300.511,272.287,300.191172,114,0.857143,...,0,0,0,0,0,0,0,0,1,4
3,12.542211,12.542211,0.175600,-0.175600,0.510163,350.549,324.341,350.137422,126,1.173913,...,0,0,0,0,0,0,0,0,2,0
4,12.817589,12.817589,0.184059,-0.184059,0.318494,432.676,404.452,432.125143,150,0.964286,...,0,0,0,0,0,0,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,5.448759,5.448759,0.613257,0.613257,0.035659,2093.634,1894.050,2091.310291,788,0.211268,...,0,0,0,0,0,0,0,0,7,52
1092,6.964899,6.964899,0.506884,-0.861916,0.035381,2097.349,1897.765,2095.457524,820,0.245033,...,0,0,0,0,0,0,0,0,2,48
1093,7.004105,7.004105,0.497358,-0.874785,0.035381,2261.603,2057.987,2259.432966,868,0.223602,...,0,0,0,0,0,0,0,0,4,48
1094,17.056452,17.056452,0.015571,0.015571,0.035580,2289.808,2066.032,2287.561538,886,0.162500,...,0,0,0,0,0,0,0,0,6,56


## 4. delete features with high correlations

In [259]:
corr_matrix = df_rdkit_screen.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
# to_drop = [column for column in upper.columns if (any(upper[column] > 0.95) or any(upper[column] < 0.05))]
to_drop = [column for column in upper.columns if (any(upper[column] >= 0.9))]
df_rdkit_screen.drop(to_drop, axis=1, inplace=True)

In [260]:
df_rdkit_screen

,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,FpDensityMorgan1,AvgIpc,BalabanJ,HallKierAlpha,Ipc,...,fr_ketone,fr_ketone_Topliss,fr_methoxy,fr_oxazole,fr_para_hydroxylation,fr_pyridine,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_thiazole
0,2.353121,1.287870,1.287870,0.608740,196.359,1.153846,2.392910,2.318302,-0.30,9.882719e+02,...,0,0,0,0,0,0,0,0,0,0
1,2.323896,1.288704,1.288704,0.485488,228.426,1.214286,2.449432,2.209145,0.05,1.636221e+03,...,0,0,0,0,0,0,1,0,0,0
2,2.313310,1.230210,1.230210,0.463323,300.511,0.857143,2.820706,1.833656,-1.08,7.084486e+04,...,0,0,0,0,0,0,0,0,0,0
3,12.542211,0.175600,-0.175600,0.510163,350.549,1.173913,3.126321,2.074957,-1.13,1.790850e+05,...,0,0,0,0,0,0,0,0,0,0
4,12.817589,0.184059,-0.184059,0.318494,432.676,0.964286,3.403667,1.756705,-1.43,2.790959e+06,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,5.448759,0.613257,0.613257,0.035659,2093.634,0.211268,1.559299,0.950806,-3.76,2.952379e+47,...,0,0,0,0,0,0,0,0,0,0
1092,6.964899,0.506884,-0.861916,0.035381,2097.349,0.245033,1.494910,0.981006,-8.22,9.598092e+54,...,0,0,0,0,0,0,0,0,0,0
1093,7.004105,0.497358,-0.874785,0.035381,2261.603,0.223602,1.494203,0.904925,-8.82,3.895594e+59,...,0,0,0,0,0,0,0,0,0,0
1094,17.056452,0.015571,0.015571,0.035580,2289.808,0.162500,1.550819,0.900316,-6.52,4.676873e+56,...,0,0,0,0,0,0,0,0,0,0


## 5. delete features low correlated with experimental optical gap

In [261]:
r_gap=0.05

In [262]:
correlations = []
for descriptor in df_rdkit_screen.columns:
    correlation, _ = pearsonr(df_rdkit_screen[descriptor], df_exp)
    if abs(correlation)>=r_gap:
        correlations.append(descriptor)
#     print(correlation)
len(correlations)

54

In [263]:
df_rdkit_screen = df_rdkit_screen[correlations]
# df_rdkit_screen.columns

df_rdkit_screen.to_csv('rdkit_feature_list.csv', index = False)

# ML regression

In [264]:
## b. metrics

def acc(y_test,y_pred):
    MSE = mean_squared_error(y_test,y_pred)
    RMSE = MSE ** 0.5
    R2 = r2_score(y_test,y_pred)
#     p = pearsonr(y_test,y_pred.reshape(-1,1)) # y_pred shape = (xxx,)
    r, p_value = pearsonr(y_test,y_pred) # y_pred shape = (xxx,)
    MAE = mean_absolute_error(y_test,y_pred)
    return RMSE, R2, r, MAE

y = df_exp
y

X = df_rdkit_screen
X

## c. model define

# model = HistGradientBoostingRegressor()
# model = LGBMRegressor(force_col_wise=True, verbose=-1)
# model = GradientBoostingRegressor()
model = XGBRegressor()
# model = AdaBoostRegressor()
# model = RandomForestRegressor()

## d. 10fold-CV plus 10fold-CV average

foldername = 'xgb-10fold-cv-plus-10fold-cv-average'
# foldername = 'hgbr-10fold-cv-plus-10fold-cv-average'
os.makedirs(foldername, exist_ok=True)

xfold=10
kf = KFold(n_splits=xfold, shuffle=True, random_state=42)

scores = []

# save index for train and test of each fold
train_idx_list = []
test_idx_list = []
i=0
for fold_idx, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    train_idx_list.append(train_index)
    test_idx_list.append(test_index)
    
    kf_sub = KFold(n_splits=xfold, shuffle=True, random_state=42)
    for fold_idx_sub, (train_index_sub, test_index_sub) in enumerate(kf_sub.split(X_train)):
        X_kf_train, X_kf_test = X_train.iloc[train_index_sub], X_train.iloc[test_index_sub]
        y_kf_train, y_kf_test = y_train.iloc[train_index_sub], y_train.iloc[test_index_sub]
        

        
        # 训练模型
        model.fit(X_kf_train, y_kf_train)

        # 获取模型预测结果
        y_pred = model.predict(X_kf_test)
        
        # 在验证集上做预测

        RMSE_test, R2_test, r_test, MAE_test = acc(y_kf_test,y_pred)

        scores.append([RMSE_test, R2_test, r_test, MAE_test])
        
        # 保存模型
        model_filename = foldername + f'/model_fold_{fold_idx + 1}_subfold_{fold_idx_sub + 1}.pkl'
        joblib.dump(model, model_filename)
        
        i+=1
        
#         print('model:',i)


scores_df = pd.DataFrame(scores, columns = ['RMSE', 'R2', 'r', 'MAE'])
index_list=[f'fold{i}' for i in range(1,xfold+1)]
index = [index_list[i%xfold] for i in range(xfold**2)]

scores_df.index = index
scores_df.round(3)

## e. load 10-fold cv plus 10-fold cv average

models = []
for fold_idx in range(xfold):
    for fold_idx_sub in range(xfold):
        model_filename = foldername + f'/model_fold_{fold_idx + 1}_subfold_{fold_idx_sub + 1}.pkl'
        model = joblib.load(model_filename)
        models.append(model)
len(models)

scores = []

for i in range(xfold):
    train_index = train_idx_list[i]
    test_index = test_idx_list[i]
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index] 
    
    predictions = []
    
    for j in range(xfold):
        model = models[i*xfold+j]     
               
        y_pred = model.predict(X_test)
        predictions.append(y_pred)
        
#         print('model idx: ',i*xfold+j)
        
    df_predictions = pd.DataFrame(predictions)
    df_predictions = df_predictions.T
    df_predictions['mean'] = df_predictions.iloc[:,:4].mean(axis = 1)
    
    RMSE_test, R2_test, r_test, MAE_test = acc(y_test,df_predictions['mean'])
    scores.append([RMSE_test, R2_test, r_test, MAE_test])


scores_df = pd.DataFrame(scores, columns = ['RMSE', 'R2', 'r', 'MAE'])
scores_df.loc['mean'] = scores_df.iloc[:xfold,:].mean()
scores_df.loc['std'] = scores_df.iloc[:xfold,:].std()
scores_df.round(3)

,RMSE,R2,r,MAE
0,0.113,0.642,0.804,0.083
1,0.126,0.615,0.787,0.090
2,0.156,0.521,0.726,0.109
3,0.129,0.654,0.817,0.086
4,0.123,0.629,0.805,0.083
5,0.148,0.431,0.657,0.103
6,0.146,0.556,0.747,0.095
7,0.138,0.433,0.659,0.085
8,0.128,0.675,0.824,0.091
9,0.129,0.498,0.709,0.091
